In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from accelerate import Accelerator
# accelerator = Accelerator()


In [ ]:
train=pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")
train=train.dropna().reset_index()
test=pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")
sub=pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")

In [ ]:
import pandas as pd
df=pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")
# .iloc[:5000]
df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
import pandas as pd


# Adding a new column based on existing column values using custom function
def custom_transformation(age):
    if age >= 4:
        return 2
    elif age==3:
        return 1
    else:
        return 0

df['binary'] = df['Score'].apply(custom_transformation)
df


In [ ]:
import pandas as pd


# Adding a new column based on existing column values using custom function
def custom_transformation(age):
    if age >= 3:
        return 1
    else:
        return 0

df['binary'] = df['Score'].apply(custom_transformation)
df

In [ ]:
label_counts = df['binary'].value_counts()
print("Number of rows for each label:")
print(label_counts)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Define the number of samples for each class
num_samples_class_1 = 486404
num_samples_class_0 = 82007

# Calculate the number of true positives, false positives, true negatives, and false negatives
true_positives = num_samples_class_1 * 0.9583
false_positives = num_samples_class_0 * 0.2173
true_negatives = num_samples_class_0 * 0.7827
false_negatives = num_samples_class_1 * 0.0417

# Create the confusion matrix
confusion_matrix_values = np.array([[true_negatives, false_positives],
                                    [false_negatives, true_positives]])

# # Plot the confusion matrix using seaborn
# plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_values, annot=True, fmt='.0f', cmap='Blues',
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
output_file = 'ternary.csv'
df.to_csv(output_file, index=False)

# EDA

In [ ]:
y=df["binary"].values
import seaborn as sns
sns.displot(y)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd



# Extract the binary column values
y = df["binary"].values

# Create a labeled frequency plot using Seaborn
sns.set(style="whitegrid")  # Set the style of the plot
plt.figure(figsize=(6, 4))  # Set the figure size
plot = sns.countplot(x=y)  # Create the count plot

# Set labels and title
plot.set(xticklabels=["Label 0", "Label 1"], xlabel="Label", ylabel="Frequency")
plt.title("Label Frequency Plot")

# Show the plot
plt.show()


In [ ]:
import re
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
import emoji
def dataPreprocessing(x):    
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("@\w+", '',x) # removing mentions (@)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    x = re.sub("http\w+", '',x)
    x = re.sub("\s[a-z]\s", '',x)
    x = x.strip()
    return x
# df["Summary"]=df["Summary"].apply(lambda x: dataPreprocessing(x))
df["Text"]=df["Text"].apply(lambda x: dataPreprocessing(x))

df

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score
from scipy.sparse import hstack
stopwords = stopwords.words('english')
stopwords = stopwords[:116]
stopwords = stopwords.extend(['d', 'll', 're', 's', 've'])
# data=pd.concat([train_text1,test_text1],axis=0).reset_index()["Summary"]
# data

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer(r'\w+')
stemmer = PorterStemmer()

# Load stopwords and extend the list
stop_words = stopwords.words('english')
additional_stop_words = ['d', 'll', 're', 's', 've']
stop_words = stop_words[:116] + additional_stop_words

# Preprocess the text data
def preprocess_text(text):
    tokens = tokenizer.tokenize(text.lower())  # Tokenize and convert to lowercase
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]  # Apply stemming and remove stopwords
    preprocessed_text = ' '.join(tokens)  # Join tokens back into a string
    return preprocessed_text

# Apply preprocessing to each row of the 'summary' column
# df["Summary"]=df["Summary"].apply(lambda x: dataPreprocessing(x))
df["Text"]=df["Text"].apply(preprocess_text)
df

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Summary"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 sentiment classes

# Tokenize the input data
encoded_train_inputs = tokenizer(xtrain, padding=True, truncation=True, return_tensors="pt")
encoded_test_inputs = tokenizer(xtest, padding=True, truncation=True, return_tensors="pt")
train_input_ids = encoded_train_inputs["input_ids"]
train_attention_mask = encoded_train_inputs["attention_mask"]
test_input_ids = encoded_test_inputs["input_ids"]
test_attention_mask = encoded_test_inputs["attention_mask"]

# Convert labels to tensors
ytrain_tensor = torch.tensor(ytrain)
ytest_tensor = torch.tensor(ytest)

# Create DataLoader for training and testing
class_weights = 1.0 / np.bincount(ytrain)
class_weights = torch.FloatTensor(class_weights)

train_dataset = TensorDataset(train_input_ids, train_attention_mask, ytrain_tensor)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, ytest_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
accelerator = Accelerator()
for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", ncols=100)
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    progress_bar.set_postfix({"Loss": total_loss / len(progress_bar)})
        
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


# GPU

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Summary"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 sentiment classes


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Tokenize the input data
encoded_train_inputs = tokenizer(xtrain, padding=True, truncation=True, return_tensors="pt")
encoded_test_inputs = tokenizer(xtest, padding=True, truncation=True, return_tensors="pt")
train_input_ids = encoded_train_inputs["input_ids"].to(device)  # Move to the same device as the model
train_attention_mask = encoded_train_inputs["attention_mask"].to(device)  # Move to the same device as the model
test_input_ids = encoded_test_inputs["input_ids"].to(device)  # Move to the same device as the model
test_attention_mask = encoded_test_inputs["attention_mask"].to(device)  # Move to the same device as the model

# Convert labels to tensors and move to the same device
ytrain_tensor = torch.tensor(ytrain, device=device)
ytest_tensor = torch.tensor(ytest, device=device)

# Create DataLoader for training and testing
class_weights = 1.0 / np.bincount(ytrain)
class_weights = torch.FloatTensor(class_weights)

train_dataset = TensorDataset(train_input_ids, train_attention_mask, ytrain_tensor)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, ytest_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
# accelerator = Accelerator()
for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", ncols=100)
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    progress_bar.set_postfix({"Loss": total_loss / len(progress_bar)})
        
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


# GPU-5lakh

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Summary"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)
# Split data into train, validation, and test sets
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.1, random_state=42)  # Split again for validation

# ... (rest of your code)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 sentiment classes


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# tokenizer.to(device)
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the first GPU
    model = model.to(device)
    model = nn.DataParallel(model, device_ids=[0, 1])  # Use both GPUs (cuda:0 and cuda:1)
model.to(device)

ytrain_tensor = torch.tensor(ytrain, device=device)
ytest_tensor = torch.tensor(ytest, device=device)

# Create DataLoader for training and testing
class_weights = 1.0 / np.bincount(ytrain)
class_weights = torch.FloatTensor(class_weights)
# Tokenize the input data
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        
        input_ids = encoding['input_ids'].squeeze().to(device)
        attention_mask = encoding['attention_mask'].squeeze().to(device)
        
        return input_ids, attention_mask, label

train_dataset = CustomDataset(xtrain, ytrain, tokenizer, max_length=128)  # Adjust max_length as needed
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = CustomDataset(xval, yval, tokenizer, max_length=128)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# Similar DataLoader setup for test data
test_dataset = CustomDataset(xtest, ytest, tokenizer, max_length=128)  # Adjust max_length as needed
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Convert labels to tensors and move to the same device


# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
# accelerator = Accelerator()
gradient_accumulation_steps = 4  # Accumulate gradients over 4 steps


train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []
train_predictions = []
for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
#     train_accuracies = []
#     progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", ncols=100)
    #progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{5}', dynamic_ncols=True)
    start_time = time.time()
    train_predictions = []
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        
        y_batch = y_batch.to(input_ids_batch.device)
        
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()  # Convert to GPU-compatible list
    
        train_predictions.extend(predicted_labels)
        loss = loss_fn(outputs.logits, y_batch)
#         loss = outputs.loss
        loss = loss.mean()  
        loss = loss / gradient_accumulation_steps  # Normalize the loss
        loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            total_loss += loss.item()
        elapsed_time = time.time() - start_time
        remaining_steps = len(train_dataloader) - step
        remaining_time = elapsed_time * remaining_steps
#         train_predictions.extend(predicted_labels.numpy())  # Extend the list of predictions
        batch_description = f'Epoch {epoch + 1}/{5}, Batch {step + 1}/{len(train_dataloader)}, Remaining Time: {remaining_time:.2f} sec'
        print(batch_description, end='\r') 
    print()
    total_loss /= (step + 1)
    train_losses.append(total_loss)
    train_accuracies.append(accuracy_score(ytrain, train_predictions))

    #progress_bar.set_postfix({"Remaining Time": f"{remaining_time:.2f} sec"})
#     progress_bar.close()
    # Training complete for this epoch
    
    
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    val_predictions = []
    #validation_accuracies = []
    with torch.no_grad():
        for step, batch in enumerate(val_dataloader):
            input_ids_batch, attention_mask_batch, y_batch = batch
            outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
            predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()  # Convert to GPU-compatible list
    
            val_predictions.extend(predicted_labels)
            y_batch = y_batch.to(outputs.logits.device)  # Move labels to the same device as the model output
            loss = loss_fn(outputs.logits, y_batch)
            val_loss += loss.mean().item()
            batch_description1 = f'Epoch {epoch + 1}/{5}, Batch {step + 1}/{len(val_dataloader)}, Remaining Time: {remaining_time:.2f} sec'
            print(batch_description1, end='\r') 
    print()
    val_loss /= len(val_dataloader)
    validation_losses.append(val_loss)
    validation_accuracies.append(accuracy_score(yval, val_predictions))
    #print(f"Epoch {epoch + 1}, Train Loss: {total_loss}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")
    
#     progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{5}', dynamic_ncols=True)
    print(f"Epoch {epoch + 1}, Train Loss: {train_losses[-1]}, Train Acc: {train_accuracies[-1]}, Validation Loss: {validation_losses[-1]}, Validation Acc: {validation_accuracies[-1]}")
    
    #print(f"Epoch {epoch + 1} training finished")
#     progress_bar.set_postfix({"Loss": total_loss / (step + 1)})
        
      
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")
# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


# better

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import time
from sklearn.utils.class_weight import compute_class_weight


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Summary"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)
# Split data into train, validation, and test sets
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.1, random_state=42)  # Split again for validation

# ... (rest of your code)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 sentiment classes


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# tokenizer.to(device)
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the first GPU
    model = model.to(device)
    model = nn.DataParallel(model, device_ids=[0, 1])  # Use both GPUs (cuda:0 and cuda:1)
model.to(device)

ytrain_tensor = torch.tensor(ytrain, device=device)
ytest_tensor = torch.tensor(ytest, device=device)

# Create DataLoader for training and testing
# class_weights = 1.0 / np.bincount(ytrain)
# class_weights = torch.FloatTensor(class_weights)
classes = [0, 1, 2]
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=ytrain)
class_weights = torch.FloatTensor(class_weights).to(device)
# Tokenize the input data
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        
        input_ids = encoding['input_ids'].squeeze().to(device)
        attention_mask = encoding['attention_mask'].squeeze().to(device)
        
        return input_ids, attention_mask, label

train_dataset = CustomDataset(xtrain, ytrain, tokenizer, max_length=128)  # Adjust max_length as needed
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = CustomDataset(xval, yval, tokenizer, max_length=128)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# Similar DataLoader setup for test data
test_dataset = CustomDataset(xtest, ytest, tokenizer, max_length=128)  # Adjust max_length as needed
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Convert labels to tensors and move to the same device


# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
# accelerator = Accelerator()
gradient_accumulation_steps = 4  # Accumulate gradients over 4 steps


train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []
train_predictions = []
for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
#     train_accuracies = []
#     progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", ncols=100)
    #progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{5}', dynamic_ncols=True)
    start_time = time.time()
    train_predictions = []
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        
        y_batch = y_batch.to(input_ids_batch.device)
        
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()  # Convert to GPU-compatible list
    
        train_predictions.extend(predicted_labels)
        loss = loss_fn(outputs.logits, y_batch)
#         loss = outputs.loss
        loss = loss.mean()  
        loss = loss / gradient_accumulation_steps  # Normalize the loss
        loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            total_loss += loss.item()
        elapsed_time = time.time() - start_time
        remaining_steps = len(train_dataloader) - step
        remaining_time = elapsed_time * remaining_steps
#         train_predictions.extend(predicted_labels.numpy())  # Extend the list of predictions
        batch_description = f'Epoch {epoch + 1}/{5}, Batch {step + 1}/{len(train_dataloader)}, Remaining Time: {remaining_time:.2f} sec'
        print(batch_description, end='\r') 
    print()
    total_loss /= (step + 1)
    train_losses.append(total_loss)
    train_accuracies.append(accuracy_score(ytrain, train_predictions))

    #progress_bar.set_postfix({"Remaining Time": f"{remaining_time:.2f} sec"})
#     progress_bar.close()
    # Training complete for this epoch
    
    
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    val_predictions = []
    #validation_accuracies = []
    with torch.no_grad():
        for step, batch in enumerate(val_dataloader):
            input_ids_batch, attention_mask_batch, y_batch = batch
            outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
            predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()  # Convert to GPU-compatible list
    
            val_predictions.extend(predicted_labels)
            y_batch = y_batch.to(outputs.logits.device)  # Move labels to the same device as the model output
            loss = loss_fn(outputs.logits, y_batch)
            val_loss += loss.mean().item()
            batch_description1 = f'Epoch {epoch + 1}/{5}, Batch {step + 1}/{len(val_dataloader)}, Remaining Time: {remaining_time:.2f} sec'
            print(batch_description1, end='\r') 
    print()
    val_loss /= len(val_dataloader)
    validation_losses.append(val_loss)
    validation_accuracies.append(accuracy_score(yval, val_predictions))
    #print(f"Epoch {epoch + 1}, Train Loss: {total_loss}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")
    
#     progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{5}', dynamic_ncols=True)
    print(f"Epoch {epoch + 1}, Train Loss: {train_losses[-1]}, Train Acc: {train_accuracies[-1]}, Validation Loss: {validation_losses[-1]}, Validation Acc: {validation_accuracies[-1]}")
    
    #print(f"Epoch {epoch + 1} training finished")
#     progress_bar.set_postfix({"Loss": total_loss / (step + 1)})
        
      
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")
# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


In [ ]:
df

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=ytrain)
class_weights

# Last

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import time
from sklearn.utils.class_weight import compute_class_weight


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Text"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)
# Split data into train, validation, and test sets
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.1, random_state=42)  # Split again for validation

# ... (rest of your code)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 3 sentiment classes


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# tokenizer.to(device)
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the first GPU
    model = model.to(device)
    model = nn.DataParallel(model, device_ids=[0, 1])  # Use both GPUs (cuda:0 and cuda:1)
model.to(device)

ytrain_tensor = torch.tensor(ytrain, device=device)
ytest_tensor = torch.tensor(ytest, device=device)

# Create DataLoader for training and testing
# class_weights = 1.0 / np.bincount(ytrain)
# class_weights = torch.FloatTensor(class_weights)
classes = [0, 1]
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=ytrain)

# class_weights = compute_class_weight(class_weight=None, classes=classes, y=ytrain)
# custom_weights=np.array([1, 1])  
# class_weights = class_weights * custom_weights
class_weights = torch.FloatTensor(class_weights).to(device)
# Tokenize the input data
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        
        input_ids = encoding['input_ids'].squeeze().to(device)
        attention_mask = encoding['attention_mask'].squeeze().to(device)
        
        return input_ids, attention_mask, label

train_dataset = CustomDataset(xtrain, ytrain, tokenizer, max_length=128)  # Adjust max_length as needed
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = CustomDataset(xval, yval, tokenizer, max_length=128)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# Similar DataLoader setup for test data
test_dataset = CustomDataset(xtest, ytest, tokenizer, max_length=128)  # Adjust max_length as needed
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Convert labels to tensors and move to the same device


# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
#loss_fn = torch.nn.CrossEntropyLoss()
# accelerator = Accelerator()
gradient_accumulation_steps = 4  # Accumulate gradients over 4 steps


train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []
train_predictions = []
for epoch in range(3):  # Example: 5 epochs
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
#     train_accuracies = []
#     progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", ncols=100)
    #progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{5}', dynamic_ncols=True)
    start_time = time.time()
    train_predictions = []
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        
        y_batch = y_batch.to(input_ids_batch.device)
        
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()  # Convert to GPU-compatible list
    
        train_predictions.extend(predicted_labels)
        loss = loss_fn(outputs.logits, y_batch)
#         loss = outputs.loss
        loss = loss.mean()  
        loss = loss / gradient_accumulation_steps  # Normalize the loss
        loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            total_loss += loss.item()
        elapsed_time = time.time() - start_time
#         remaining_steps = len(train_dataloader) - step
#         remaining_time = elapsed_time * remaining_steps
#         train_predictions.extend(predicted_labels.numpy())  # Extend the list of predictions
        batch_description = f'Epoch {epoch + 1}/{3}, Batch {step + 1}/{len(train_dataloader)}, Elapsed Time: {elapsed_time:.2f} sec'
        print(batch_description, end='\r') 
    print()
    total_loss /= (step + 1)
    train_losses.append(total_loss)
    train_accuracies.append(accuracy_score(ytrain, train_predictions))

    #progress_bar.set_postfix({"Remaining Time": f"{remaining_time:.2f} sec"})
#     progress_bar.close()
    # Training complete for this epoch
    
    
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    val_predictions = []
    #validation_accuracies = []
    start_time = time.time()
    with torch.no_grad():
        for step, batch in enumerate(val_dataloader):
            input_ids_batch, attention_mask_batch, y_batch = batch
            outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
            predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()  # Convert to GPU-compatible list
    
            val_predictions.extend(predicted_labels)
            y_batch = y_batch.to(outputs.logits.device)  # Move labels to the same device as the model output
            loss = loss_fn(outputs.logits, y_batch)
            val_loss += loss.mean().item()
            elapsed_time = time.time() - start_time
            
            batch_description1 = f'Epoch {epoch + 1}/{3}, Batch {step + 1}/{len(val_dataloader)}, Elapsed Time: {elapsed_time:.2f} sec'
            print(batch_description1, end='\r') 
    print()
    val_loss /= len(val_dataloader)
    validation_losses.append(val_loss)
    validation_accuracies.append(accuracy_score(yval, val_predictions))
    #print(f"Epoch {epoch + 1}, Train Loss: {total_loss}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")
    
#     progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{5}', dynamic_ncols=True)
    print(f"Epoch {epoch + 1}, Train Loss: {train_losses[-1]}, Train Acc: {train_accuracies[-1]}, Validation Loss: {validation_losses[-1]}, Validation Acc: {validation_accuracies[-1]}")
    
    #print(f"Epoch {epoch + 1} training finished")
#     progress_bar.set_postfix({"Loss": total_loss / (step + 1)})
        
      
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")
# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


In [ ]:
7.05
7.31
7.56
8.15
8.31
8.58
9.26
9.55


In [ ]:
gff

In [ ]:
plt.figure(figsize=(12, 4))

# Plot Losses
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(validation_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

# Plot Accuracies
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(validation_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Example true labels and predicted labels


# Compute confusion matrix
cm = confusion_matrix(ytest, all_predictions)

# Create a DataFrame from the confusion matrix
confusion_matrix_df = pd.DataFrame(cm, index=['0', '1', '2'], columns=['0', '1', '2'])

# Set color palette for the heatmap
#cmap = sns.color_palette("Blues")
base_color = "#3D59AB"  # Specify the base color
num_shades = 920  # Specify the number of shades you want

# Create a list of shades of the base color
shades = sns.light_palette(base_color, n_colors=num_shades)

# Create a custom color map
cmap = sns.color_palette(shades)

# Plot the confusion matrix using a heatmap
sns.heatmap(confusion_matrix_df, annot=True, cmap='Blues', fmt='')
#annot_kws={"size": 10}
# Add labels and title
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Text"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 3 sentiment classes

# Check if GPUs are available
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the first GPU
    model = model.to(device)
    model = nn.DataParallel(model, device_ids=[0, 1])  # Use both GPUs (cuda:0 and cuda:1)

ytrain_tensor = torch.tensor(ytrain, device=device)
ytest_tensor = torch.tensor(ytest, device=device)

# Create DataLoader for training and testing
class_weights = 1.0 / np.bincount(ytrain)
class_weights = torch.FloatTensor(class_weights)

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        
        input_ids = encoding['input_ids'].squeeze().to(device)
        attention_mask = encoding['attention_mask'].squeeze().to(device)
        
        return input_ids, attention_mask, label

train_dataset = CustomDataset(xtrain, ytrain_tensor, tokenizer, max_length=128)  # Adjust max_length as needed
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Similar DataLoader setup for test data
test_dataset = CustomDataset(xtest, ytest_tensor, tokenizer, max_length=128)  # Adjust max_length as needed
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
gradient_accumulation_steps = 4  # Accumulate gradients over 4 steps

for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{5}', dynamic_ncols=True)
    start_time = time.time()
    for step, batch in enumerate(progress_bar):
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        loss = loss / gradient_accumulation_steps  # Normalize the loss
        loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            total_loss += loss.item()
        elapsed_time = time.time() - start_time
        remaining_steps = len(train_dataloader) - step
        remaining_time = elapsed_time * remaining_steps
        progress_bar.set_postfix({"Remaining Time": f"{remaining_time:.2f} sec"})

    # Training complete for this epoch
    print(f"Epoch {epoch + 1} training finished")
        
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


# Past

In [ ]:
jgjgjgj

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Summary"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 sentiment classes

# Tokenize the input data
encoded_train_inputs = tokenizer(xtrain, padding=True, truncation=True, return_tensors="pt")
encoded_test_inputs = tokenizer(xtest, padding=True, truncation=True, return_tensors="pt")
train_input_ids = encoded_train_inputs["input_ids"]
train_attention_mask = encoded_train_inputs["attention_mask"]
test_input_ids = encoded_test_inputs["input_ids"]
test_attention_mask = encoded_test_inputs["attention_mask"]

# Convert labels to tensors
ytrain_tensor = torch.tensor(ytrain)
ytest_tensor = torch.tensor(ytest)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_input_ids, train_attention_mask, ytrain_tensor)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, ytest_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


In [ ]:
accumulation_steps = 4  # Accumulate gradients over 4 batches

for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    accumulated_loss = 0
    
    for i, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        
        # Accumulate gradients
        loss = loss / accumulation_steps
        accumulated_loss += loss
        
        if (i + 1) % accumulation_steps == 0 or i == len(train_dataloader) - 1:
            accumulated_loss.backward()
            optimizer.step()
            total_loss += accumulated_loss.item()
            accumulated_loss = 0

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Summary"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 sentiment classes



encoded_train_inputs = tokenizer(xtrain, padding=True, truncation=True, return_tensors="pt")
encoded_test_inputs = tokenizer(xtest, padding=True, truncation=True, return_tensors="pt")
train_input_ids = encoded_train_inputs["input_ids"]
train_attention_mask = encoded_train_inputs["attention_mask"]
test_input_ids = encoded_test_inputs["input_ids"]
test_attention_mask = encoded_test_inputs["attention_mask"]

# Convert labels to tensors
ytrain_tensor = torch.tensor(ytrain)
ytest_tensor = torch.tensor(ytest)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_input_ids, train_attention_mask, ytrain_tensor)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, ytest_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ... (Rest of your data processing and training loop)

# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        input_ids_batch = input_ids_batch.to(device)
        attention_mask_batch = attention_mask_batch.to(device)
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# ... (Rest of the code remains unchanged up to the DataLoader setup)

# Compute class weights for reweighting
class_weights = 1.0 / np.bincount(ytrain)
class_weights = torch.FloatTensor(class_weights)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_input_ids, train_attention_mask, ytrain_tensor)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, ytest_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ... (Rest of the code remains unchanged for the training loop and evaluation)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)  # Apply class weights

for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Evaluation
# ... (Rest of the code remains unchanged for the evaluation)


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Load data from CSV
# data = pd.read_csv("your_data.csv")  # Replace with your CSV file

# Assuming the CSV has columns "text" and "label"
texts = df["Summary"].tolist()
labels = df["binary"].tolist()  # Make sure the label column is encoded as integers (0, 1, 2)

# Split data into train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 sentiment classes

# Tokenize the input data
encoded_train_inputs = tokenizer(xtrain, padding=True, truncation=True, return_tensors="pt")
encoded_test_inputs = tokenizer(xtest, padding=True, truncation=True, return_tensors="pt")
train_input_ids = encoded_train_inputs["input_ids"]
train_attention_mask = encoded_train_inputs["attention_mask"]
test_input_ids = encoded_test_inputs["input_ids"]
test_attention_mask = encoded_test_inputs["attention_mask"]

# Convert labels to tensors
ytrain_tensor = torch.tensor(ytrain)
ytest_tensor = torch.tensor(ytest)

# Create DataLoader for training and testing
class_weights = 1.0 / np.bincount(ytrain)
class_weights = torch.FloatTensor(class_weights)

train_dataset = TensorDataset(train_input_ids, train_attention_mask, ytrain_tensor)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, ytest_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Evaluation
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_mask_batch, _ = batch  # Labels are not needed for inference
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(ytest, all_predictions)
print("Test Accuracy:", test_accuracy)


In [ ]:
train=train.iloc[:1000]
test=test.iloc[:1000]
# sub=sub.iloc[:10000]
# sub=sub.iloc[-10000:]
# sub

In [ ]:
y=df["Score"].values
import seaborn as sns
sns.displot(y)

In [ ]:
import pandas as pd


# Adding a new column based on existing column values using custom function
def custom_transformation(age):
    if age < 4:
        return 0
    else:
        return 1

df['binary'] = df['Score'].apply(custom_transformation)

print(df)


In [ ]:
output_file = 'ternary.csv'
df.to_csv(output_file, index=False)

# Start

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV data into a DataFrame
# data = pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv').iloc[:1000]
# data = pd.read_csv('/kaggle/working/ternary.csv')
# .iloc[:50000]
data = df

# Define your features (X) and target variable (y)
X = data
# y = data['Score']
y = data['binary']
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
train=X_train[["Summary","binary"]]
train

In [ ]:
ytrain=X_train[["binary"]]
ytrain

In [ ]:
train=train[['Summary']]
train

In [ ]:
test_text=X_test["Summary"]
test_text

In [ ]:
xtrain1

In [ ]:
import numpy as np
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Sample data
xtrain1 = train
ytrain = ytrain

# Load pre-trained RoBERTa model and tokenizer
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming 2 labels

# Tokenize the input data
encoded_inputs = tokenizer(xtrain1, padding=True, truncation=True, return_tensors="pt")
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]

# Convert labels to tensor
ytrain_tensor = torch.tensor(ytrain)

# Define a DataLoader
dataset = TensorDataset(input_ids, attention_mask, ytrain_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(5):  # Example: 5 epochs
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids_batch, attention_mask_batch, y_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Inference
model.eval()
test_text=test_data
with torch.no_grad():
#     Assuming you have a separate test set, preprocess it similarly
    encoded_test_inputs = tokenizer(test_data, padding=True, truncation=True, return_tensors="pt")
    test_input_ids = encoded_test_inputs["input_ids"]
    test_attention_mask = encoded_test_inputs["attention_mask"]

#     Run predictions on test data
    test_outputs = model(test_input_ids, attention_mask=test_attention_mask)
    predicted_labels = torch.argmax(test_outputs.logits, dim=1)


In [ ]:
import re
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
import emoji
def dataPreprocessing(x):    
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("@\w+", '',x) # removing mentions (@)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    x = re.sub("http\w+", '',x)
    x = re.sub("\s[a-z]\s", '',x)
    x = x.strip()
    return x
train_text1=train["Summary"].apply(lambda x: dataPreprocessing(x))

In [ ]:
test_text1=test_text.apply(lambda x: dataPreprocessing(x))

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
print(device)

In [ ]:
# import torch

# # Check if at least one CUDA-enabled GPU is available
# if torch.cuda.is_available():
#     # Determine the number of available GPUs
#     num_gpus = torch.cuda.device_count()
#     print(f"Number of available GPUs: {num_gpus}")

#     # Select GPUs for training (0 and 1 in this case)
#     device_ids = [0, 1]  # Specify the GPU indices you want to use
#     device = torch.device("cuda:0")  # This is just a placeholder since DataParallel will handle device placement
#     # Uncomment the next line to use both GPUs for a single operation
#     # device = torch.device("cuda" if len(device_ids) > 1 else "cuda:0")
# else:
#     device = torch.device("cpu")

# print("Selected device:", device)

# # Example usage of torch.nn.DataParallel
# # model = YourModel()  # Instantiate your model here
# # model = torch.nn.DataParallel(model, device_ids=device_ids)
# # model.to(device)  # Move the model to the selected device


In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score
from scipy.sparse import hstack
stopwords = stopwords.words('english')
stopwords = stopwords[:116]
stopwords = stopwords.extend(['d', 'll', 're', 's', 've'])
data=pd.concat([train_text1,test_text1],axis=0).reset_index()["Summary"]
data

In [ ]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stopwords,
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(data)
train_word_features = word_vectorizer.transform(train_text1)
test_word_features = word_vectorizer.transform(test_text1)

In [ ]:
xtrain2 = hstack([train_word_features])
xtest2 = hstack([test_word_features])

In [ ]:
import torch.nn as nn

In [ ]:
from torch import nn
from transformers import RobertaTokenizer, RobertaModel
from transformers import AdamW
from tqdm import tqdm
class BertClassificationModel(nn.Module):
    def __init__(self,hidden_size=768,num_class=3): 
        super(BertClassificationModel, self).__init__()
        model_name = "siebert/sentiment-roberta-large-english"
        self.tokenizer = RobertaTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
        self.bert = RobertaModel.from_pretrained(pretrained_model_name_or_path=model_name)

        for p in self.bert.parameters(): 
                p.requires_grad = False
        self.fc = nn.Linear(hidden_size, num_class)

    def forward(self, batch_sentences): 
        sentences_tokenizer = self.tokenizer(batch_sentences,
                                             truncation=True,
                                             padding=True,
                                             max_length=512,
                                             add_special_tokens=True)
        input_ids=torch.tensor(sentences_tokenizer['input_ids']).to(device) 
        attention_mask=torch.tensor(sentences_tokenizer['attention_mask']).to(device) 
        bert_out=self.bert(input_ids=input_ids,attention_mask=attention_mask) 

        last_hidden_state =bert_out[0].to(device) 
        bert_cls_hidden_state=last_hidden_state[:,0,:].to(device) 
        return bert_cls_hidden_state
model=BertClassificationModel()
model=model.to(device)

In [ ]:
import torch


In [ ]:
# if torch.cuda.is_available():
#     device = torch.device('cuda')
#     model = model.to(device)
#     model = nn.DataParallel(model, device_ids=[0, 1])

In [ ]:
# if torch.cuda.is_available():
#     # Determine the number of available GPUs
#     num_gpus = torch.cuda.device_count()
#     print(f"Number of available GPUs: {num_gpus}")

#     # Select GPUs for training (0 and 1 in this case)
#     device_ids = [0, 1]  # Specify the GPU indices you want to use
#     device = torch.device("cuda:0")  # This is just a placeholder since DataParallel will handle device placement
#     # Uncomment the next line to use both GPUs for a single operation
#     # device = torch.device("cuda" if len(device_ids) > 1 else "cuda:0")
# else:
#     device = torch.device("cpu")

# print("Selected device:", device)

In [ ]:
model=model.to(device)

In [ ]:
xtrain1=[]
for i in tqdm(train_text1):
    xtrain1.append(model([i]).cpu().detach().numpy())
xtrain1=np.array(xtrain1).reshape(-1,768)  
del train_text1

In [ ]:
xtrain1=np.array(xtrain1).reshape(-1,204800)  
del train_text1

In [ ]:
xtrain1=[]
for i in tqdm(train_text1):
    xtrain1.append(model([i]).cpu().detach().numpy())
xtrain1=np.array(xtrain1)#.reshape(-1,819200)  
del train_text1

In [ ]:
xtrain1

In [ ]:
import csv

my_array = xtrain1

with open('xtrain1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Index', 'Value'])  # Writing header
    for index, value in enumerate(my_array):
        writer.writerow([index, value])


In [ ]:
nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

In [ ]:
# xtrain1 = []
# for i in tqdm(train_text1):
#     xtrain1.append(model([i]).to(device).detach().cpu().numpy())
# xtrain1 = np.array(xtrain1).reshape(-1, 768)
# del train_text1

In [ ]:
import numpy as np

In [ ]:
xtest1=[]
for i in tqdm(test_text1):
    xtest1.append(model([i]).cpu().detach().numpy())
xtest1=np.array(xtest1).reshape(-1,768)
del test_text1

In [ ]:
xtest1=[]
for i in tqdm(test_text1):
    xtest1.append(model([i]).cpu().detach().numpy())
xtest1=np.array(xtest1).reshape(-1,204800)
del test_text1

In [ ]:
import csv

my_array = xtest1

with open('xtest1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Index', 'Value'])  # Writing header
    for index, value in enumerate(my_array):
        writer.writerow([index, value])


In [ ]:
ytrain=train["binary"].values
import seaborn as sns
sns.displot(ytrain)

In [ ]:
ytrain.size

In [ ]:
sub=pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")
sub=sub.iloc[-50000:]

In [ ]:
sub=sub["Score"].values
import seaborn as sns
sns.displot(sub)

In [ ]:
ytrain=ytrain-1
ytrain

In [ ]:
sub=sub-1
sub

In [ ]:
ytrain

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
#class_weight = 'balanced'
classes=[0,1,2]
weight = compute_class_weight(class_weight='balanced', classes=classes, y=ytrain)
print(weight)

In [ ]:
# from sklearn.utils.class_weight import compute_class_weight
# # class_weight = 'balanced'
# classes=[0,1,2,3,4]
# weight = compute_class_weight(class_weight='balanced', classes=classes, y=sub)
# print(weight)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Define your classes and corresponding weights
# classes = [0, 1, 2, 3, 4]
custom_weights = weight  # Define the weights for each class
custom_weights=np.array(custom_weights)
classes=np.array(classes)


# Compute class weights using the custom weights
ytrain_array = np.array(ytrain)

class_weights = compute_class_weight(class_weight=None, classes=classes, y=ytrain_array)
class_weights *= custom_weights  # Multiply with custom weights

print("Custom Class Weights:", class_weights)
# class_weight_dict = {cls: weight for cls, weight in zip(classes, custom_weights)}

# # Compute class weights using the custom weights
# class_weights = compute_class_weight(class_weight=class_weight_dict, classes=classes, y=ytrain)

# print("Custom Class Weights:", class_weights)


In [ ]:
class_weights=np.array(class_weights)

In [ ]:
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error 
from xgboost import XGBClassifier
xgb_1=XGBClassifier(tree_method='gpu_hist',
                    predictor='gpu_predictor',n_jobs=-1)
rmse_scores=[]
kf = KFold(n_splits = 2)
for train_ix, test_ix in tqdm(kf.split(xtrain1)):
    X_train, X_test = xtrain1[train_ix], xtrain1[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    xgb_1.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=Y_train))
    pred = xgb_1.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
a=np.mean(rmse_scores)
print(a)
xgb_1.fit(xtrain1,ytrain,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=ytrain))

In [ ]:
from sklearn.metrics import accuracy_score  # Import accuracy_score
from sklearn.utils import class_weight


In [ ]:
class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

print("Custom Class Weights:", class_weight_dict)

In [ ]:
xtrain1.size

In [ ]:
xtest1.size

In [ ]:
train_text1

In [ ]:
from torch import nn
from transformers import RobertaTokenizer, RobertaModel
from transformers import AdamW
from tqdm import tqdm
class BertClassificationModel(nn.Module):
    def __init__(self,hidden_size=768,num_class=3): 
        super(BertClassificationModel, self).__init__()
        model_name = "siebert/sentiment-roberta-large-english"
        self.tokenizer = RobertaTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
        self.bert = RobertaModel.from_pretrained(pretrained_model_name_or_path=model_name)

        for p in self.bert.parameters(): 
                p.requires_grad = False
        self.fc = nn.Linear(hidden_size, num_class)

    def forward(self, batch_sentences): 
        sentences_tokenizer = self.tokenizer(batch_sentences,
                                             truncation=True,
                                             padding=True,
                                             max_length=512,
                                             add_special_tokens=True)
        input_ids=torch.tensor(sentences_tokenizer['input_ids']).to(device) 
        attention_mask=torch.tensor(sentences_tokenizer['attention_mask']).to(device) 
        bert_out=self.bert(input_ids=input_ids,attention_mask=attention_mask) 

        last_hidden_state =bert_out[0].to(device) 
        bert_cls_hidden_state=last_hidden_state[:,0,:].to(device) 
        return bert_cls_hidden_state
model=BertClassificationModel()
model=model.to(device)

In [ ]:
from transformers import RobertaForSequenceClassification
#xtrain, ytrain,xtest, ytest = train_test_split(train_text1, test_text1, test_size=0.2, random_state=42)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
ytrain_tensor = torch.tensor(ytrain)
ytest_tensor = torch.tensor(ytest)


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_encodings = tokenizer(xtrain, truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(xtest, truncation=True, padding=True, return_tensors="pt")
# Training loop
for epoch in range(5):  # Example: 5 epochs
    model.train()
    optimizer.zero_grad()
    train_outputs = model(**train_encodings, labels=ytrain_tensor)
    loss = train_outputs.loss
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(**test_encodings)
    predicted_classes = torch.argmax(test_outputs.logits, dim=1)
    accuracy = accuracy_score(ytest, predicted_classes)

print("Accuracy:", accuracy)

In [ ]:
from xgboost import XGBClassifier
xgb_1=XGBClassifier(tree_method='gpu_hist',
                    predictor='gpu_predictor',n_jobs=-1)
accuracy_scores=[]
kf = KFold(n_splits = 3)
for train_ix, test_ix in tqdm(kf.split(xtrain1)):
    X_train, X_test = xtrain1[train_ix], xtrain1[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    xgb_1.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight=class_weight_dict, y=Y_train))
    pred = xgb_1.predict(X_test)
    accuracy = accuracy_score(Y_test, pred)  # Calculate accuracy for each fold
    print(accuracy)
    accuracy_scores.append(accuracy)

mean_accuracy = np.mean(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)

# Train the final model on the entire dataset
xgb_1.fit(xtrain1, ytrain, sample_weight=class_weight.compute_sample_weight(class_weight=class_weight_dict, y=ytrain))

In [ ]:
from xgboost import XGBClassifier
xgb_1=XGBClassifier(tree_method='gpu_hist',
                    predictor='gpu_predictor',n_jobs=-1)
xgb_1.fit(xtrain1, ytrain, sample_weight=class_weight.compute_sample_weight(class_weight=class_weight_dict, y=ytrain))

In [ ]:
xgb_1.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight=class_weight_dict, y=Y_train))


In [ ]:
pred = xgb_1.predict(X_test)
accuracy = accuracy_score(Y_test, pred)

In [ ]:
print(accuracy)

In [ ]:
X_train, X_test = xtrain1[train_ix], xtrain1[test_ix]
Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
xgb_1.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight=class_weight_dict, y=Y_train))
pred = xgb_1.predict(X_test)
accuracy = accuracy_score(Y_test, pred)
print(accuracy)

In [ ]:
xgb_2=XGBClassifier(tree_method='gpu_hist',
                    predictor='gpu_predictor',n_jobs=-1)
rmse_scores=[]
for train_ix, test_ix in tqdm(kf.split(xtrain2)):
    X_train, X_test = xtrain2[train_ix], xtrain2[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    xgb_2.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=Y_train))
    pred = xgb_2.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
b=np.mean(rmse_scores)
print(b)
xgb_2.fit(xtrain2,ytrain,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=ytrain))

In [ ]:
from sklearn.linear_model import LogisticRegression
logic_1=LogisticRegression()
rmse_scores=[]
for train_ix, test_ix in tqdm(kf.split(xtrain1)):
    X_train, X_test = xtrain1[train_ix], xtrain1[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    logic_1.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=Y_train))
    pred = logic_1.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
c=np.mean(rmse_scores)
print(c)
logic_1.fit(xtrain1,ytrain,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=ytrain))

In [ ]:
logic_2=LogisticRegression()
rmse_scores=[]
for train_ix, test_ix in tqdm(kf.split(xtrain2)):
    X_train, X_test = xtrain2[train_ix], xtrain2[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    logic_2.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=Y_train))
    pred = logic_2.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
d=np.mean(rmse_scores)
print(d)
logic_2.fit(xtrain2,ytrain,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=ytrain))

In [ ]:
ypred1=xgb_1.predict_proba(xtest1)
y1=xgb_1.predict(xtest1)
sns.displot(y1)

In [ ]:
ypred2=xgb_2.predict_proba(xtest2)
y2=xgb_2.predict(xtest2)
sns.displot(y2)

In [ ]:
ypred3=logic_1.predict_proba(xtest1)
y3=logic_1.predict(xtest1)
sns.displot(y3)

In [ ]:
ypred4=logic_2.predict_proba(xtest2)
y4=logic_2.predict(xtest2)
sns.displot(y4)

In [ ]:
from lightgbm import LGBMClassifier
lgb_1=LGBMClassifier(device="gpu",
                   gpu_platform_id=0,
                   gpu_device_id=0)
rmse_scores=[]
for train_ix, test_ix in tqdm(kf.split(xtrain1)):
    X_train, X_test = xtrain1[train_ix], xtrain1[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    lgb_1.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=Y_train))
    pred = lgb_1.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
e=np.mean(rmse_scores)
print(e)
lgb_1.fit(xtrain1,ytrain,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=ytrain))

In [ ]:
lgb_2=LGBMClassifier(device="gpu",
                   gpu_platform_id=0,
                   gpu_device_id=0)
rmse_scores=[]
for train_ix, test_ix in tqdm(kf.split(xtrain2)):
    X_train, X_test = xtrain2[train_ix], xtrain2[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    lgb_2.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=Y_train))
    pred = lgb_2.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
f=np.mean(rmse_scores)
print(f)
lgb_2.fit(xtrain2,ytrain,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=ytrain))

In [ ]:
from catboost import CatBoostClassifier
cat_1=CatBoostClassifier(logging_level='Silent',
                         task_type='GPU',loss_function='MultiClass')
rmse_scores=[]
for train_ix, test_ix in tqdm(kf.split(xtrain1)):
    X_train, X_test = xtrain1[train_ix], xtrain1[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    cat_1.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=Y_train))
    pred = cat_1.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
g=np.mean(rmse_scores)
print(g)
cat_1.fit(xtrain1,ytrain,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=ytrain))

In [ ]:
cat_2=CatBoostClassifier(logging_level='Silent',
                         task_type='GPU',loss_function='MultiClass')
rmse_scores=[]
for train_ix, test_ix in tqdm(kf.split(xtrain2)):
    X_train, X_test = xtrain2[train_ix], xtrain2[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    cat_2.fit(X_train,Y_train,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=Y_train))
    pred = cat_2.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
h=np.mean(rmse_scores)
print(h)
cat_2.fit(xtrain2,ytrain,sample_weight=class_weight.compute_sample_weight(class_weight='balanced', y=ytrain))

In [ ]:
ypred5=lgb_1.predict_proba(xtest1)
y5=lgb_1.predict(xtest1)
sns.displot(y5)

In [ ]:
ypred6=lgb_2.predict_proba(xtest2)
y6=lgb_2.predict(xtest2)
sns.displot(y6)

In [ ]:
ypred7=cat_1.predict_proba(xtest1)
y7=cat_1.predict(xtest1)
sns.displot(y7)

In [ ]:
ypred8=cat_2.predict_proba(xtest2)
y8=cat_2.predict(xtest2)
sns.displot(y8)

In [ ]:
!pip install pytorch_tabnet

In [ ]:
w=ytrain.copy()
w=w.astype("float32")
for i in range(ytrain.shape[0]):
    w[i]=weight[ytrain[i]]
w

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
tab_1=TabNetClassifier()
rmse_scores=[]
for train_ix, test_ix in tqdm(kf.split(xtrain1)):
    X_train, X_test = xtrain1[train_ix], xtrain1[test_ix]
    Y_train, Y_test = ytrain[train_ix], ytrain[test_ix]
    ww=w[train_ix]
    tab_1.fit(X_train,Y_train,weights=ww,eval_metric=['mae'])
    pred = tab_1.predict(X_test)
    rmse_scores.append(mean_absolute_error(Y_test,pred))
i=np.mean(rmse_scores)
print(i)
tab_1.fit(xtrain1,ytrain,weights=w,eval_metric=['mae'])

In [ ]:
ypred9=tab_1.predict_proba(xtest1)
y9=tab_1.predict(xtest1)
sns.displot(y9)

In [ ]:
y=np.argmax((ypred1/a+ypred2/b+ypred3/c+ypred4/d+ypred5/e+ypred6/f+ypred7/g+ypred8/h+ypred9/i)/(1/a+1/b+1/c+1/d+1/e+1/f+1/g+1/h+1/i),axis=1)+1
sns.displot(y)

**And also you can try regressor to do,because we use MAE to judge our model actually**

In [ ]:
sub["Rating"]=y
sub

In [ ]:
sub.to_csv("sample_submission.csv",index=False)